# Tools and Routing

Previously, we showed how functions can be supplied to OpenAI models. 

They can be used to specify an output schema required for tasks (e.g., extraction, tagging) or for an API. 

Tools are more general and are a central concept in [LLM agents](https://python.langchain.com/docs/use_cases/more/agents/).

Often times, tools wrap APIs so that LLMs can easily access them. 

We can define tools (e.g., from functions) as shown below.

In [1]:
from langchain.agents import tool

In [2]:
@tool
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [3]:
search.name

'search'

In [4]:
search.description

'search(query: str) -> str - Search for the weather online.'

In [3]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

We can specify the tool input.

And, tools can be suppied as functions. 

**Note: May need to explain how this works.**

In [3]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

from langchain.tools.render import format_tool_to_openai_function
format_tool_to_openai_function(search)

/Users/swang294/anaconda3/envs/genie/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'search',
 'description': 'search(query: str) -> str - Search for the weather online.',
 'parameters': {'type': 'object',
  'properties': {'query': {'description': 'Thing to search for',
    'type': 'string'}},
  'required': ['query']}}

In [3]:
search.run("sf")

'42f'

In [5]:
!pip freeze | grep langchain

langchain==0.0.291
langchain-community==0.0.24
langchain-core==0.1.27
langchain-experimental==0.0.29


In [5]:
! pip install langchain --upgrade

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 15.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 23.8 MB/s eta 0:00:00
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.7
    Uninstalling orjson-3.9.7:
      Successfully uninstalled orjson-3.9.7
  Attempting uninstall: langsmith
    Found existing installation: langsm

### Weather Tool


In [8]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params, verify=False)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [6]:
get_current_temperature.name

'get_current_temperature'

In [ ]:
get_current_temperature.description
### make some notes

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [9]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [10]:
get_current_temperature({"latitude": 13, "longitude": 14})

/Users/swang294/anaconda3/envs/genie/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.open-meteo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'The current temperature is 34.6°C'

### Wikipedia Tool

In [8]:
! pip install wikipedia

In [8]:
import os
os.environ['CURL_CA_BUNDLE'] = ""
os.environ['PYTHONWARNINGS']="ignore:Unverified HTTPS request"
SSL_CERT_DIR= '/Users/swang294/Library/CloudStorage/OneDrive-JNJ/Logistics/zscaler_cert/'

In [11]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [12]:
wikipedia.summary("Wikipedia")

SSLError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=Wikipedia&srinfo=suggestion&format=json&action=query (Caused by SSLError(SSLCertVerificationError(5, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [10]:
search_wikipedia.name

'search_wikipedia'

In [11]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [ ]:
wikipedia.search("Barack")

In [ ]:
search_wikipedia.run({"query": "George Washington"})

# Querying an API from a OpenAPI specfiction


In [14]:
! pip install openapi_schema_pydantic

In [15]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [16]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "0.28.1",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [ ]:
spec = OpenAPISpec.from_text(text)

In [ ]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [91]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [92]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [93]:
model.invoke("what are three pet names")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'listPets', 'arguments': '{\n  "params": {\n    "limit": 3\n  }\n}'}})

In [94]:
model.invoke("tell me about pet with id 43")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'showPetById', 'arguments': '{\n  "path_params": {\n    "petId": "43"\n  }\n}'}})

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
functions = [format_tool_to_openai_function(f) for f in [search_wikipedia, get_current_temperature]]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [26]:
model.invoke([HumanMessage(content="What is the weather in san francisco right now?")])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

In [27]:
model.invoke([HumanMessage(content="What is langchain")])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{\n  "query": "langchain"\n}'}})

In [28]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [29]:
chain.invoke({"input": "What is the weather in san francisco right now?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

In [30]:
chain.invoke({"input": "hi!"})

AIMessage(content='Hello! How can I assist you today?')

In [31]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [32]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [33]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [34]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [35]:
result.tool

'get_current_temperature'

In [36]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [37]:
get_current_temperature(result.tool_input)

'The current temperature is 28.7°C'

In [38]:
result = chain.invoke({"input": "hi!"})

In [39]:
type(result)

langchain.schema.agent.AgentFinish

In [40]:
result.return_values

{'output': 'Hello! How can I assist you today?'}

In [41]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [42]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [43]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [44]:
result

'The current temperature is 28.7°C'

In [45]:
result = chain.invoke({"input": "What is langchain?"})

In [46]:
result

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which encodes meaningful semantic information.State of the art embeddings are based on the learned hidden layer representation of dedicated sentence transformer models. BERT pioneered an approach involving the use of a dedicated [CLS] token preprended to the beginning of each sentence inputted into the model; the final hidden state vector of this token encodes information about the sentence and can be fine-tuned for use in sentence classification tasks. In practice however,

In [47]:
result = chain.invoke({"input": "Hi!"})

In [48]:
result

'Hello! How can I assist you today?'